# **Day12**: *Hot Springs*

## Part 1

You finally reach the hot springs! You can see steam rising from secluded areas attached to the primary, ornate building.

As you turn to enter, the researcher stops you. "Wait - I thought you were looking for the hot springs, weren't you?" You indicate that this definitely looks like hot springs to you.

"Oh, sorry, common mistake! This is actually the onsen! The hot springs are next door."

You look in the direction the researcher is pointing and suddenly notice the massive metal helixes towering overhead. "This way!"

It only takes you a few more steps to reach the main gate of the massive fenced-off area containing the springs. You go through the gate and into a small administrative building.

"Hello! What brings you to the hot springs today? Sorry they're not very hot right now; we're having a lava shortage at the moment." You ask about the missing machine parts for Desert Island.

"Oh, all of Gear Island is currently offline! Nothing is being manufactured at the moment, not until we get more lava to heat our forges. And our springs. The springs aren't very springy unless they're hot!"

"Say, could you go up and see why the lava stopped flowing? The springs are too cold for normal operation, but we should be able to find one springy enough to launch you up there!"

There's just one problem - many of the springs have fallen into disrepair, so they're not actually sure which springs would even be safe to use! Worse yet, their condition records of which springs are damaged (your puzzle input) are also damaged! You'll need to help them repair the damaged records.

In the giant field just outside, the springs are arranged into rows. For each row, the condition records show every spring and whether it is operational (.) or damaged (#). This is the part of the condition records that is itself damaged; for some springs, it is simply unknown (?) whether the spring is operational or damaged.

However, the engineer that produced the condition records also duplicated some of this information in a different format! After the list of springs for a given row, the size of each contiguous group of damaged springs is listed in the order those groups appear in the row. This list always accounts for every damaged spring, and each number is the entire size of its contiguous group (that is, groups are always separated by at least one operational spring: #### would always be 4, never 2,2).

So, condition records with no unknown spring conditions might look like this:

```
#.#.### 1,1,3
.#...#....###. 1,1,3
.#.###.#.###### 1,3,1,6
####.#...#... 4,1,1
#....######..#####. 1,6,5
.###.##....# 3,2,1
```

However, the condition records are partially damaged; some of the springs' conditions are actually unknown (?). For example:

```
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
```

Equipped with this information, it is your job to figure out how many different arrangements of operational and broken springs fit the given criteria in each row.

In the first line (???.### 1,1,3), there is exactly one way separate groups of one, one, and three broken springs (in that order) can appear in that row: the first three unknown springs must be broken, then operational, then broken (#.#), making the whole row #.#.###.

The second line is more interesting: .??..??...?##. 1,1,3 could be a total of four different arrangements. The last ? must always be broken (to satisfy the final contiguous group of three broken springs), and each ?? must hide exactly one of the two broken springs. (Neither ?? could be both broken springs or they would form a single contiguous group of two; if that were true, the numbers afterward would have been 2,3 instead.) Since each ?? can either be #. or .#, there are four possible arrangements of springs.

The last line is actually consistent with ten different arrangements! Because the first number is 3, the first and second ? must both be . (if either were #, the first number would have to be 4 or higher). However, the remaining run of unknown spring conditions have many different ways they could hold groups of two and one broken springs:

```
?###???????? 3,2,1
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#
```

In this example, the number of possible arrangements for each row is:


- ???.### 1,1,3 - 1 arrangement
- .??..??...?##. 1,1,3 - 4 arrangements
- ?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
- ????.#...#... 4,1,1 - 1 arrangement
- ????.######..#####. 1,6,5 - 4 arrangements
- ?###???????? 3,2,1 - 10 arrangements

Adding all of the possible arrangement counts together produces a total of 21 arrangements.

For each row, count all of the different arrangements of operational and broken springs that meet the given criteria. What is the sum of those counts?

### Solution

We can use dynamic programming, specifically dynamic functions, in order to tackle this problem. 

In [1]:
from typing import List
from functools import wraps

In [2]:
def memoize(func):
    """
    A wrapper that stores the output of a function with certain args and kwargs
    in a cache dictionary.
    
    https://www.youtube.com/watch?v=qORqpMg3Uew
    """
    cache = {}

    @wraps(func)
    def wrapper(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]

    return wrapper


In [3]:
@memoize
def evaluate_record(record:str, numbers:List[int]):
    """
    A recursive function that counts the number of possible combinations.
    
    The function returns 1 only if the entire record has been processed and all
    of the `numbers` have been used. 
    
    If there are characters left in the record there are three options
    
        -   The record starts with '.', in which case the number of possible 
            combinations of the rest of the record is unchanged; we can simply 
            evaluate the record with the first character removed
    
        -   The record starts with '#', in which case we need to check if it is 
            possible to form n consecutive '#'
            
        -   The record starts with '?', in which case we will simply evaluate
            the function for both a '.' and '#'; only valid combinations will
            return `1`
    """
    # entire record processed and all numbers used
    if len(record) == 0: 
        return 1 if len(numbers)==0 else 0
    
    # only one possible combination
    if record.startswith('.'): 
        return evaluate_record(record[1:], numbers)

    # check if it is possible to form n consecutive '#'
    if record.startswith('#'):
        if len(numbers)==0: return 0
        
        n = numbers[0]
        if len(record) < n: return 0        # record too small
        if '.' in record[0:n]: return 0     # not enough consecutive '#'
        
        if len(record) > n:
            if record[n] == '#': return 0   # too many consecutive '#'
            if len(numbers) > 1: return evaluate_record(record[n+1:], numbers[1:])  # n+1 because we know the next character has to be '.', which can be skipped 
            
        return evaluate_record(record[n:], numbers[1:])
        
    # check both '.' and '#'
    if record.startswith('?'):
        combination1 = evaluate_record(record.replace('?', '.', 1), numbers)
        combination2 = evaluate_record(record.replace('?', '#', 1), numbers)
        return combination1 + combination2
    



In [4]:
evaluate_record('?###????????', numbers=[3, 2, 1])


10

In [5]:
def count_arrangements(input_string):
    lines = input_string.splitlines()
    records = [line.split(' ')[0] for line in lines]
    inputs = [list(map(int, line.split(' ')[1].split(','))) for line in lines]

    combinations = []
    for record, numbers in zip(records, inputs):
        combinations.append(evaluate_record(record, numbers))
    
    return sum(combinations)
    

### Example

In [6]:
input_string = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""


count_arrangements(input_string)


21

### Submission

In [7]:
input_string = """???#...#.?#??#?#? 1,1,1,8
#?.???????#????#???. 1,1,12
????.#??##??? 1,2,3
??#.#????#???#.??. 2,10,1
??????.?.???##?#?. 6,1,7
?#????????????###?? 1,3,9,1
?.??#?#?#.???? 7,1
.????????..??.#?. 5,2,2
.?.??#???.. 1,1,1
???#?????????#?? 2,1,3,2
??#?#?????. 4,2
?####???.???#? 6,1,2
#?.?#?#?????.?#.?.? 1,5,2,1,1,1
??.##?#?##??.??? 1,7,1
???.###?#?# 1,7
?????##?#..#? 1,3,1,1
??#??#?.???.?#??#? 1,1,1,2,1,2
?#?#?.???.?? 4,1,1
.?#?????#?? 3,1,2
?#??????????.??? 3,7,1
???????##??###???#?? 2,3,5,4
?.???#.??? 1,1,1
???.#?#?????.?#???#? 2,5,1,3,3
?.#?.#????.??## 1,2,1,3
?????????##?#??????? 1,5,9,1
?###????#??##?#... 9,4
????.#?#????..???. 5,1
????#??.##??#?#??.?? 1,2,5,3
..?????#.???.??##? 1,1,2,1,3
?#?????#??##.??# 12,1
?#??#?#.#????? 1,4,2,2
???#?...???????.? 3,2,4
??#??#?#?#?????#?.#. 1,1,12,1
???#??..?###. 5,4
?#.??#?#????????# 2,2,1,7
????.?????. 1,2,1
?.??????#??.???#.?? 1,1,2,1,4,1
???#??????.??????. 6,1,1,1,1,1
?#??#?#??##??? 1,4,5
?..??#??????.??. 1,4,1,1,2
?.??##???..??? 1,3,1,1
?##???.????#?? 3,3
??????.??#????? 1,1,5
?##.????#????.#??. 2,8,1
????#????#??# 1,10
???.?#??????#???.? 1,2,1,1,1,2
.?###.##??? 4,2
??.????#???#???###.? 1,15,1
?#????#?#???. 3,1,1,4
???????.????#???? 6,4
..??..???# 2,1
.?.?#?????..????..? 1,4,1,1,2,1
?#?.?.???????? 3,1,1,5
?###?.??#??.?###?# 4,3,6
..?#?.??#??##? 3,2,2
.?????#?????#?##?? 1,3,2,5
?#???#??.?.#?????? 8,2,3
??.??????#?#???? 1,2,5,1
?.??????#? 1,3,1
?...#??.?????#? 1,3,6
?.??.?..?#???? 1,1,1,4
.#??????.? 1,1
?????..?#????#.? 1,2,2,4,1
#.###?#????. 1,5,1,1
.???????#.???#?. 1,1,3,1,1
?#?#?.???????. 4,2
#????????#? 8,1
#?.?#????????? 1,2,6
?#??#????##?.???.? 5,1,3,2,1
????#????#???#?.???? 1,10,1,4
?##??????.???#?..?? 9,1,2,1
?#.??#???.. 1,1,2
.?#?????#.? 3,3,1
??.????.#??? 1,2
#??#??.????#??.?.??? 1,1,1,7,1,1
##????.???. 2,2
??##?.?##? 3,3
?#?..?.??..??.?? 1,1
?#????????? 3,1,2
?##??.???.?.?# 3,1,1,2
?.####???????? 1,4,1,1
?..#.???#????? 1,7
?.??.??##???? 1,1,4,1
.#?#?????#?.#??# 1,1,1,1,4
??.?????#????.??##. 8,4
????????.??.##? 1,1,2,1,3
??##???#???#?#??.? 6,1,3
?..??#???#??. 1,8
????????#????#??#??. 2,11,1
?.???????????#?#. 1,3,1,1,4
?#?#.??#.???#???##?? 1,1,2,11
??#?#???#?#??????? 11,1
.#.??#?????#? 1,6,1
..?????????#?. 3,2
????.??.#.?#?#?????. 1,2,1,1,4,2
#??????????##?.??? 5,1,1,3,1,1
????????..???? 3,2,3
??.????#??#???# 1,7,1
..??#.?????????? 1,5
?#?#???#??#??##?#??? 1,3,1,2,8
?????#.?#???.???# 2,1,4,1,1
?.?#??.??.????#??#? 1,8
??#??.?????? 3,3
?#?..??.?#? 2,1,3
.????##???. 1,6
???...?.????##?? 2,7
???..?###?#? 3,6
??.?#?#??###?????#? 1,4,8,1
.????#??#?# 1,6,1
?????.??.? 1,2
?#???##??.?..### 1,5,1,3
.???#???#??. 4,3
???#?.?#????. 1,5
?????..??.#?#?###??# 1,2,1,10
????#??????.?#?..#?? 5,2,1,3,2
#????#???????#???#.? 6,10,1
????#??????.?.?..?. 9,1
.??????##????.???#.? 2,8,2,1
.??##?.???#?# 1,3,1,3
????##??##??#?#. 1,11,1
???#?##.?.??? 4,1
?????#?#??? 1,6
?...????????.? 1,2
#??????##??..??? 2,1,4,1
.????????#??. 3,1,1
??#??????##..?.????? 3,6,3
??###??.??###?. 6,5
???.???#???.#?.?? 1,6,1,1
?????#??????? 1,6
.#??#?#?#??.??? 2,7,1,1
#??..??????#? 1,1,1,5
?#..?##??#?. 1,7
?#?#?#??..?.?. 7,1,1
.????????????.?##??? 1,1,8,6
??????####?..# 2,6,1
?????#???#???????? 10,1,1
??#??#???.??#??? 3,1,2,1,3
.#..?.?.?###??#...# 1,1,1,6,1
?#?#???.#??#????..? 1,3,4,1,1
.?##???#????#???? 3,2,4,1
###?..#??#??? 4,1,1,1
???.??#??#??. 1,7
????#?##??? 1,1,5
?.????????????##? 1,1,3,1,4
??#????#??. 1,4,1
.??#???#?.???#???#?? 8,6
##???#??.???#?.?? 6,1,1,1,2
#?#??#???#? 3,2,3
??#??????#??#?? 2,6,1
?.????#?#..??????.?? 6,2
.??.??.#?#?##? 1,1,1,4
????...?????????? 3,5
??#####.###???.?? 7,3,2
?##.??#??#???#??? 2,11
??????.??.?.???? 3,1,1,1,1
?.##?????##?? 2,4
?.???#?????.??????? 1,5,1,1,1,1
?????????#??? 2,9
?#???#?.#?.???.# 2,1,2,1,1
?#??????#?#? 1,1,1,3
?#??????#??.#????? 9,1,1,1,1
#???.?.?#????#???# 4,1,2,5,1
??#??????? 1,6
?????.?#?? 1,1,2
?????##.?.?# 1,3,1,1
???#?????#??# 1,2,2,4
.?#??????..?# 4,1,1
.#???.?.??##.???? 3,1,4,4
???.???#.????.#????? 3,5
#??#??#... 4,1
??.??#??#..???.?? 5,1
??.?#?????..##????? 1,6,2,1,1
..?#??#???????#?#? 8,5
?.??#??.?.??????. 3,6
.#??###???????##??. 1,6,5
??????#???#? 1,3,1,1
??#.?##?.?? 1,1,4
.??#???.?#????.?. 6,2,1,1
.??????#??#?? 6,2
#.?#??##?.??? 1,6,1
.?.##?????. 1,2,1
.?#?.????# 2,5
.????#?.#? 1,2,1
?.???.####???#.?. 1,8
?.#?????.??.???#?? 1,1,1,1,1,6
??#.?#???##??#. 3,9
??.#???????.???#???? 1,1,1,1,2,4
??.???????##? 2,2,3,2
.?.??#..??.. 1,1,1,2
????#?.???###?#???# 4,1,6,1,1
????#??????...???? 4,3,1
#?.#?.?????#??? 2,1,1,4
.????????#???#??.?#? 2,2,8,1
?????.##?? 2,3
##??.##???? 2,4
?????#??#?#?.#??.?? 10,3,2
?????..??????#?. 4,3,4
..?#.?#?#???. 1,5
..?.?##??? 1,2,2
#??#???##?? 1,3,3
??.#?????.????. 1,2
?##?????#?? 6,1,1
????#??###? 3,4
??????#???????#? 6,5,2
?#.??.?#?.#????#.? 1,1,2,1,4
?????#?#?###???##?? 1,5,3,6
#???#?#?#??????.?? 10,2
?.??#???.#.?#?.?#?? 1,5,1,2,3
?#??##?#.????.. 7,3
??#..??.?#?? 3,2,1
???#??.???.?? 5,1,1,1
??.??#.?#.??.??? 1,2,2,1,1
??.#???#?. 1,2,1
??.??##???. 1,5
?##?#????#??.???.# 3,1,5,1,1
??###?###?#????.???? 1,13,1,1
.???.??##.? 1,4
??????.??###??????? 2,5,5
.#?#?#???#?? 1,7
????#?#..#??.? 3,1,1,1
??#?.##?##?###???? 1,11
.?.#??????##???? 1,1,1,4,1
#??.##??#???#?#???#? 1,15
?????#???#?#?.? 1,8
?#???##??..?## 7,3
????????#? 1,4
##????#???????? 2,4,1,2
?#?##.?###??..?.? 4,4,1
??##???##?.??? 8,2
.???????#.?##??### 7,8
#????????????.##?? 6,1,2,3
?.??????#?????..#? 1,2,1,1,1,2
????#??..?.#???#???? 6,1,1,2,2
.??????#?.?#??.? 3,2,2
?????.??##? 3,1,4
.?.???##.##?### 5,6
?##?????##???#?.?.#? 3,1,8,1,2
????.?##.?? 3,2
?.??.??.??#?###??? 1,1,1,1,6
?.??????#?# 2,3
??#?#?.?????# 6,1,1,1
?.#?#??#?? 1,2,2
??????.#???# 2,1,3
?.#..???#. 1,1,1
#???.????? 4,1
??#?????##???.??? 10,2
??.??#.??? 1,1,1
??..???????#??? 1,1,1,1,2
#??.????#?? 1,2,1
.#?.#???.?.? 1,1,1,1
??????.##?#?. 3,5
?##???????.?##??#?? 3,2,1,8
??#?????.?#?#. 4,4
.???????#??#???.?# 12,1
.#??????.??#?.? 3,2,2,1
?????#????.??#???.?? 10,5
??.#..??#? 1,1,2
?????#?.??#? 2,2,2
?.#???.#????..??#? 1,1,1,2,2,2
?##??..#.#?.?#. 4,1,1,1
?????#.??.#??? 1,1,1,3
??????#??.???##??.?. 8,5
?#????#??????.??? 3,2,2,1
#??????????##..??.?? 1,1,1,5,1,1
##??#????????..?.?.? 13,1
.????.#??#??.?#??# 1,1,1,3,2,1
.?##??#????...?????. 10,5
##.???????????.#??? 2,1,6,1,2,1
.??#??#?.?#???? 3,1,4
.???.??????## 1,2,3
????????#??#?#?. 5,7
???.??#??#?. 2,2,1
..??????##. 3,2
??????#????????? 1,5,1
##?#???###?#?.. 10,2
???..???##?.?#..?? 2,1,3,1,2
?????##??? 1,5
?.????#?.??#? 1,5,4
..????#??.?.#?? 1,4,1
#????#?.?.. 6,1
.?.#??.?#???#???? 2,5
.?#..??.????? 2,2,2,2
???.??#????.?.??#??# 1,4,1,6
.#???????..#????? 5,5
##???.??#?? 2,1,5
?##?????#??? 5,1,3
??.#.#?????#??? 1,1,1,1,3
??.??.?#?? 1,1,4
?????.?#?? 1,3
????#?#????? 1,8
????????#?### 5,5
.??.?.??#?? 1,1,1
????????#???#?. 3,3,2,2
??.#???..?.?.?. 3,1
???????#.?????? 5,5
.???#?.????. 4,1,1
.??#??#??.?# 4,1,1
.????..?.?? 3,1,2
#??#?#?#####.?? 2,1,7,1
.?#??#???.? 2,1,2
#???#?#??#??.? 1,2,1,3
?#?#.???#? 1,1,4
??.????#?..??.?.?#? 1,1,4,2,1,2
#??????#??.???##??. 2,7,1,5
?#???.#.?????#?#? 4,1,6
###???.??..###?#?.? 5,2,6
??#?#??##?????# 1,1,5,4
.##??######???#?? 3,7,2,1
?#.?????.#?#?#???? 1,4,7
??.?#???##?????#. 1,3,6,1
??????#???????#??. 1,2,1,2,3
.???.?#?????.?.? 1,3,1,1,1
?.##??????? 1,3,2
..????.??? 1,1
..??#?.??? 4,1,1
?##???#.#? 3,2,2
???#?????.#???? 2,1,1,3,1
.??.?????..# 2,1,1,1
.????????#??? 1,3
??.???#??????? 2,4,3
?..?#?????.?##.# 7,3,1
??.?#?#????? 1,9
?#..???#?#??#.? 1,6
?#?#?????.#??. 8,2
??????????. 1,4
.?#????#?.#?## 2,1,1,4
#?#?###????.???????. 11,1,1
.#??####.?#??? 7,4
.???.#???#?###?? 2,1,1,4
#??????#?? 2,1,3
?.?.##?#?.?..#? 1,2,1,1,1
???#?#???. 4,4
?.???.??#??? 1,1,3,1
???????..## 2,1,2
?.???????.?? 1,1,5,1
#???????.?#?#??##?? 3,1,1,6
?.??.?.???. 2,2
#??##???#??.??. 5,2,1
????????#? 3,1,1
.?????#??##?.#?##??. 8,5
..??..????#?? 1,5
?????#.?#???##??? 1,1,2,8
???#....?# 1,2
?.??????#??#?????? 1,1,7,2,1
?##?.#??.. 2,2
?.?.?..??#?#??? 1,1,1,4,1
.##?.#.#??#??. 2,1,1,2
.????????#?? 2,2,3
?..#????.????#?#?#. 1,1,3,3,3,1
?.#?.?????#???????# 2,12
????##?#????#?#. 12,1
?.?#???????.? 1,2,1,1
##?.????.? 3,2,1
.??#?#???#.??.???? 6,1,2
.???????###?.????. 1,1,5,1,2
..#?????????????? 8,1,1,1
??.#??#?.?# 2,1,1
???????????#????? 2,6
?#.??.???. 1,3
?#???.?.??.??????? 1,1,1,2,6
??????#?????.?? 5,1,2,1
.#???.?.???#? 2,1,3
??##.??.???.??? 3,1,1,2
???.????#??#??##??? 3,1,11
????..##?..?? 3,2
??????????? 1,1,5
???.?#.?.?#?#??????# 3,1,1,8
.??.??#??? 2,6
##?..??????? 3,5
??????#????#.?#???# 1,1,3,1,3,1
?#?????.#?. 3,1,1
?????.??#??##??? 1,1,7,1
.????#??#? 1,4,1
#???#??#????.??? 2,9,2
?#????#??.???.?.? 1,6,1,1,1
????????.#? 2,1,1
???#??##.??#??#?. 7,4
#??.????.??????#? 3,1,1,2,2
?##?#??..?#. 3,1,1,2
.?.?????##???#.?. 3,6
#.?????### 1,2,3
????.??.?# 2,2,1
?????.????.#???#?.? 1,2,2,6,1
????????#?#.????? 4,2,1,1,2
?##????.#? 3,1,2
????.??????? 3,1,1,1
???????#?.? 4,4
.#.??????????. 1,3,4
?????#?.?#.? 2,3,2,1
?#.?????##??.???#? 1,7,1,1,2
#?????.????#?? 1,2,4
???????.????#?? 5,1,2
??.??.##?????.??#?? 1,6,5
???.?????..?##???? 2,6
???????###??##?#? 1,1,12
????????.? 1,1,1
??..???????#? 2,7
??#???.#??##?????? 4,1,5,1,1
.?.???????????#?? 2,5
?????##?.? 1,3
?????.#.??#??#?##?# 3,1,3,6
???.????#? 3,1,1
.??.???.#??.#?##???. 2,3,1,7
..?????###?#???????. 11,2
????????.???????#??# 1,5,3,2,1
??????.##?##?.? 2,5
??.??#???#???.?.? 3,5
?#?#.??#????#.??# 3,2,1,1,1
????#??.##?????.#?. 6,4,1,1
?.?.??.???? 1,2,2
#.?????????#? 1,8
.?????????#??????? 1,3,3,4
???.??????.#.??? 1,1,1,1,3
???.?#?#??? 3,4
????????.???##.??? 1,1,1,1,3,1
??.?.#.?.????.?.? 1,1
???###?##???# 1,6,3
?.?.????##??. 1,1,4
?.????.??.###???? 1,1,1,4,1
?#??????.?????#???. 1,1,8
??????#?.? 1,3
####??#?..#?????# 7,1,2
.????#??.#.???? 1,1,1,2,1
??..#??.?#?##? 3,6
?.??.??##???#????#? 5,7
??????#??.?.??? 3,2,2
?.???..?????#??.. 1,1,8
..?????##?? 2,3
??????.?????#??? 1,1,1,8
????#...?#.?? 1,3,2
??#??.??#??#?? 2,8
#???????..? 1,1,3
?#.??#.?#??# 1,3,5
#???????????###??#?# 2,1,1,1,9
.??##?.?????? 4,5
.????.??.?#. 1,2
#??????????#?..? 2,4,3,1
#?.???###?##????.?? 2,10
??#?..??????. 1,3
#??#?????#?? 1,3,5
??..?#..??.?? 1,2,2,1
?.??.????.# 1,4,1
#????.???? 2,1,2
.????#??#?????#.? 7,2
?.?.??????? 1,3,1
#?#?###??##?#???.#?? 11,2,2
#?..?????#???? 2,2,3
?.??????.????.?# 1,3,1,1,1
??????#?????.? 1,1,3,1
???#????.?#????. 4,1
##?####?#?#?.???? 11,2
??.#?.?.?#?.? 1,1
????#????###?#????? 2,1,11,1
#??#?.?#????#?#????# 4,2,1,5,1
??.???#.??#.#?#?#??# 1,2,1,1,1,8
?#?????????.?? 1,5,1,2
???#????#???#..? 4,1,2,1,1
???..??##?# 1,5
??.?.#?#??#???#?# 1,1,4,3,3
?#??#?##?#???? 5,4,1
????##??#? 4,1
#?.?????.#?.? 2,3,1,1
??.?#????????? 2,4,1,2
??.?.???.?????#? 1,1,2,2
??.????.??? 3,3
??.??????##???? 1,3
?#????.??##? 2,1,3
??#?##??#?#? 1,4,1,1
.?.#???.#.???#??#?? 1,1,1,1,8
???#.??..#??????.?? 3,1,5,1,1
?##??.#?.??.?? 2,2,2
???#?.???.?##?? 4,1,1,2
.?????????#. 3,1,3
???###?.#.#?#?.#? 6,1,4,1
????#???#?? 2,7
??#??#????#?? 7,3
..?.?????? 1,2,1
?#?#??#??. 1,4,1
??##????.? 5,1,1
.???#?##?#??? 1,1,2,4
??#???.?##??. 4,4
??##?#?#???.#?????. 4,1,4,3,1
.????##?????#?? 1,1,2,6
?.##????.#?#?.#?? 1,4,3,1
?????#?#?? 1,6
.#?##??????.?##.??. 6,1,3,1
??#?.?????.?#??.? 3,1,3
.?????###?? 2,4,1
?.?#?#.??...#..?? 1,4,1,1,1
??.????.??????#? 2,4,1,3
.????.?##?#?? 2,3,2
#?#?####?#???.#????? 12,5
??#.?????? 2,3,1
.#???#??#???#?#?..?? 1,1,1,7,1
??#????#???.#? 7,2,1
#?#?#..#???#??? 5,1,2,1
.?#?#####??.? 8,1
???#?#?.??#???.?# 6,3,2
.??#???#??? 1,2,3
???#???#??#? 2,3,1
?????.??????.?.?? 2,3,1,1
.?##????.????? 5,3
##?.#??#?. 3,2,2
???.#?##?#?#??.#?? 2,1,6,1,1
??#????.??????#?#?? 6,5,1
??#?#?...???# 4,1,1
??.?#??.?##.?.#??? 3,3,4
#?#????.??????#?? 7,1,4
.??????#??? 2,5
#?#?..????????????? 4,6,2,1
..#?#????#?? 3,4
#??#?#??????#??#?#? 10,7
?????.?#?#??? 1,6
??###???.???#?.## 6,2,1,2
????#??#?##. 3,1,4
#????#??#?????#?.?? 1,8,2,1,2
#.???#?????.? 1,4,1,1
#?.??.?????#?##.? 1,1,4,4,1
.??.???????? 1,1,1,1
??.?##?????.. 1,6
??#???.???? 1,1,1
.?.?.??##?????#?? 1,1,10,1
?????????????? 9,1
??#.#.??#???? 1,1,5,1
.???????#?#????.?.#? 5,4,2,2
.#??#??#?.? 1,5
?##?#??##??#??#?.??? 15,1
?#?#????????#?.?.# 3,3,2,1,1,1
???#??.?????#??.. 3,6
???#??..???#?.?.??? 1,2
##???#.?.. 3,1,1
??.??#?#?#???##?#?.? 2,1,1,1,9
.#????#?#????#?#?. 9,4
???#?###.##. 1,1,3,2
????#.?..???#?? 1,3
.#.????#???.#??? 1,6,1,1,1
??#.?.?#???.#?..#?. 2,1,2,1,1,2
?#????..?? 2,1
????.?#???#.???? 3,5,3
??????????##.???#.#? 1,3,1,2,4,1
?.?#.????#???##.? 1,8
#?.#????.? 1,3
.#???#.?#?????.?? 1,3,5,1
?.#????##?? 1,3,3
##?.?.#.??????#?? 3,1,1,4,1
????.??.?.????? 1,1,1,1,2
.?#?##.??#???? 4,6
?.?????.?#.. 2,1,1
????.?#??# 1,1,1
#.####??#?.#??# 1,8,1,1
.??.##???? 1,2,1
??..???.#. 2,1,1
?..????.??? 1,2
.??????#...?.????? 1,1
???.??#???##?? 1,3,4
????#??#?#??????##?? 1,5,1,1,5
?#???..???#???#??? 2,6
??????.??##?. 2,3
??#?????##??#.#?#??# 3,7,6
???.?????##?? 1,7
.?#??#?.?.?? 1,1,1,1
??.????#?? 1,2,1
???###.#???? 4,2,1
?.?#????.???????.. 3,1,6
???.??.#?#..??##. 1,1,1,1,4
????#????#.?##? 2,1,2,3
??#?##?#??.??????? 8,3
..#???#????.??? 1,3,1,2
??#?.?.???????.#.?? 3,1,1,5,1,1
?????#??.?? 1,2,1
????????..?#??#??.## 1,1,2,6,2
.#??????##??.?.??.?? 9,1,1,1
??##??#?????. 8,2
#.?.?????? 1,2
.????#?#???#..??# 2,7,3
?.?#?????????.#? 1,1,1,3,1
..#????.?????? 3,1,1,2
??#??#?.????.???.? 3,2,1,1,2
#???#.??.##??# 3,1,5
?.?.?#?##.???? 1,1,4,2
.?###..??#??????## 4,1,4
???#??????#. 2,4
??####????#?.??.. 7,1,1
#?.??????#? 1,2,1
.?##?#????????#?.??? 3,1,1,1,5,3
?.???#?##?.. 2,4
????#???..?##?# 1,1,1,3,1
??#??.#?.?????.? 4,1,4,1
???#.???.??. 4,1,2
?#?#...??#. 2,1,3
????#.?#..??. 1,2,2,2
?.#??#????#?##??? 12,1
??#??????#? 6,3
???.????#??? 1,1,1,2
???#.????.????? 1,2,5
??.?.??#??.??#??? 1,1,5,1,3
??#??????? 2,2
?.??#???????##??? 1,6,3
???#?#?.???#????#?? 7,1,8
.????????????? 3,1,2,1
?????##?#?#??.... 1,8,1
.?##???????#.???. 3,2
???.???????? 1,1,1,1
???#.?????????#?#..? 1,1,1,9,1
?##???#????.??#?#? 10,4
#??.????????? 3,2,1,1
?#?..???#?#?? 3,1,6
?##?#.?##?. 4,4
?#.#?..?#???? 1,2,3,1
??.##??#??#?#? 2,5,4
????##?#???.#.??... 8,1,1
##???????..?.?#??.? 2,4,2
?.?#.?#???. 1,4
.?#.??.????? 1,1,1,1
??????.???.?#?##??.? 2,4
???.????.??#??.?? 1,1,2,4,2
###?#??#.???????? 8,3
???..??#.????. 2,1,1
???.?##???.??????# 2,6,4
.?#?##.?#?????.?? 5,4,1
.??..??.?#? 2,2
.?#??????.##?#? 4,2,2,2
???##.?.?.#???. 2,1,1
?????#???#? 7,1
.#?.???#?.??# 1,3,1,1
?.#?..?#??#####?# 2,10
??#?##???#?#???#?#? 1,8,1,1,1,1
??????#.??#?###. 2,1,6
?#?#?#???#??##???# 1,3,9,1
?#???.????#.??#?#?? 1,1,2,2,3,3
?#?#.#.??.?#??????? 3,1,2,3,1,2
.?#?...?????#.??? 3,4,1,2
?????????#?#?.?# 1,1,1,5,1
##?#?#.???????#?? 6,4
?#??#?????????# 3,1,4,2
???#???#????.?? 1,6,1,1,1
?#.????##?.? 2,6
?.???##??.?#??#? 3,4
??#?????#.?#?. 2,1,1,3
?.?????#?#??#.?.#??? 11,2,1
????#???#?? 1,3,1
???.?#.???.???. 1,1,2,1,1
??????#?..??#? 5,2
.?.?#.?#??. 1,4
?#.??????##????.???? 2,7,1,1,1
?#?#?.?????. 3,1,1
???.##.?.#?. 3,2,1,2
..????????#.####??? 6,5
??#.??..?##?##????#? 1,1,9,1
?#???.??##?.??????. 5,4,3
.?...??#?????#.# 1,7,1,1
??????.?#??.?.? 5,2,1
???..??##?????? 2,5,1
???##?#??????.#.??. 7,2,1,1,1
?????????#?? 1,2,1,1
??.##??.?### 1,2,1,4
??####?#?###..?????? 1,10,1,1,1
??.??#???#??? 1,2,4
????..???#???. 1,6
??????????#? 8,1
#?#.?..????? 1,1,1,2
.?????##???#??? 12,1
##??#?..?#..? 2,2,1,1
###???????. 5,1,1
??.?.##?##??#?.?# 1,1,8,2
???####?.????#?#???# 1,5,10
??.??#???.?????#?#?? 1,1,4,6
???.#??#??..??.?# 2,4,1,2
.?.??##?###.???##? 8,2
.#??.?#.??#?? 3,1,3
????#????#??#??#??? 5,8
#..#???.???#???. 1,1,1,1,3
?.##?????.?#?????. 6,4,1
.????????.??#?? 2,1,1,1,1
.?.????????##?.?.?#? 1,1,5,3
??.#???????## 1,1,1,2
??????#??#???#??#??? 3,1,9,1
??.#??.???#?. 3,4
????.???##. 3,5
#??.??#????? 2,2,1
#?????#??.# 1,6,1
????.??#????.??. 1,7,1
##???????????. 6,1
.??#?????.????#???? 5,1,1,3,1
?????#??#??.?? 1,8
???#..??.???.. 1,2
???#?.#..????#?# 2,1,1,1,1
????.?.??#??#?? 2,1,7
.???#?##????#?? 2,1,4,1
??#?????#?? 4,1
.#????#?###?. 2,7
..????#??? 3,1,1
####???#..??. 6,1
?#???#???##????#?#?. 2,1,10
??#?#?#?.?????# 1,6,1,1,1
?????.##?#?#?#?.??#? 1,2,9,1
.?.????.?. 1,1,1
?????????? 1,3,1
#????#?????? 1,3,3
#.?#..???? 1,1,1
?#?????..????#.?? 1,2,4,1
#?##??????.??. 1,2,3,2
???#??#??. 5,1
?##???.?.??#.?# 3,1,1,1,2
?#?????????? 2,7
??#?#???#?#?????? 2,5,1,1,1
..#?..?.##?.? 2,2,1
???????.???.#?##.?.? 1,4,3,4,1,1
??##??###.# 7,1
?###?#??.#.??..?? 4,3,1,1,2
??????#??#?. 2,2,2
?#?##??????..#?? 6,2,3
?#?#???????..#.#?? 9,1,1,1,1
#?#?#?#????.?? 1,5,2,1
?????#??.?#? 5,1,1
??##???.?#??#.??.? 5,5,1,1
.?#?.?##?? 3,2
.???????#??..?#?? 8,1
?????.???# 4,1
?#?....???..??#? 3,2,1,1
???####?????.???#??? 1,5,1,1,1,1
???.??????...#??? 1,5,3
????#??.???????? 1,1,1,1,3
?#.???.#### 1,3,4
?.?#???#?#?#???.? 1,10,1
???.??#.?.? 2,1
??.??.??????.??.?? 1,1,3,1,1,1
.#??.?#????.#?###? 3,4,1,1,3
#?.???#?.?. 1,2,1
???#???.?? 4,2
????.#?#?#.?#####?# 3,1,3,6,1
..??????##?#?????#?? 2,1,5,2,4
#??.??#????#???#???? 1,15
???????????.? 1,5,1
?#??#???#?? 5,3
.???##????.?????.#. 1,4,1,1,1,1
?.?#?#??.?#?????? 6,2,2
.????????????? 7,1
...?.#???????#? 1,1,2,1
?????.?#?? 2,1
?#.??????? 2,2
#????#???##? 1,6,2
.?#?.?.???#?# 3,1,4
?#??#.??.?#?? 1,1,3
????#.?????#?##.?? 5,1,6,2
???..???????.##?? 2,6,2
..?#???.#?? 2,2
???##?.??#????????. 4,2,2
????#?????.?#??#. 1,5,2,1,2
##.??..#?..?#???? 2,2,1,3,1
??????#???#?#??? 5,7
.?#??.#??#???.#.#? 4,1,1,1,1,1
?.?????#?#?.?? 1,5
?????????#?????#?# 10,2,1,1
??????#??##????#??? 1,5,2,2,2,1
.??#?#????.??### 1,1,1,1,4
?????#?????#????? 4,2,2,1
??.?.?#??????#.??? 1,2,3,1,1
.??.?.???? 2,1,1
.??#????????? 1,2,3
???#?????#??#????#? 1,1,1,5,1,2
?????##???#?? 3,4,1
##?????????# 5,1,1,1
???#??#??????.? 4,1,2
????.??#?#?? 1,1,4
#??????.#???. 1,4,4
????##.??#???#?.??? 3,5
????.?#??#.##???? 1,1,4,2,3
#?#???#??????#. 1,6,2,1
???.???##?#? 1,1,4
?????#.???? 5,3
?#??.????#?????# 1,1,5,3
????#?.?#?.?.?. 1,1,2,1,1
???.???#?#?#??? 1,9
.?##??.??#???#???#? 3,3,6
?????##??.?? 1,6
#????????#####.? 2,1,7,1
#?..??.??#?? 2,1,1,3
?##?#???#???.#??? 5,3,1,1
.?????????????????. 1,1,2,1,1,4
?.???#??.?#?????? 4,5,1
?.????#??#?? 1,1,1,1
??????.#???# 3,1,3,1
#?#??..???##?#?.? 4,6
#??#?.????.?.? 5,1,1
?.?????.??#?#?##??? 1,1,1,10
???#?#.???##???.?.? 1,3,7,1,1
#?#.?#??.??? 3,3,3
.?#???#???.???# 7,3
?#????#??. 3,3
????####?#?.???? 2,5,1,2
?#???#??...??.? 2,2,2
?????????????#?###? 2,11
??.???????###?# 1,4,4,1
????#??.?.???.#???? 6,1,1,1,1,1
??#?#???????? 6,1
#??#?#????#???.? 9,4
?.#?#?#????????#??.? 8,1,3
.?.???#???????????? 1,7,1,2,2
?#???###????.??#?# 1,5,2,5
??#?????.??# 1,2,3,2
?##????.??.?# 4,1,2,1
#.?#?#???? 1,1,5
????.##??.?#??????? 3,4,4,3
?#.??????#??? 1,1,1,3
?#?.#?..??.??? 1,2,1,3
..???????? 3,3
?#????????. 1,3,2
.?#.???#.?? 2,2
?.??????????.?? 3,3,1
?#.??????..?#? 1,4,3
????????.??.#??? 6,2,2,1
?#???????.?????? 2,4,5
???.?#???. 1,1,2
??????#.?##??????#?# 1,5,3,6
??#?????#??.???#??. 4,2,1,5
.?#??.?.?. 2,1
#??#?.#?.#??.???? 5,1,1,1,1
???#????.#.? 2,1
??????#???.#.?#??##? 3,1,5
????#???????.# 9,1,1
????????#?##?#..#?. 1,2,3,2,1,2
??.?##?##.??????#? 2,2,2,4,2
###??#???#??? 3,1,3,1
.?.?.?#?.???#?# 1,1,3,1,1
.?..?#?????.???#??#? 1,2,1,1,2,3
?#.?????????#??#?#?? 2,1,14
?##.#???#?????? 2,1,1,1,1
???#.##???.? 3,5
???????.???#?? 4,1,2
??.#?????#??? 1,10
?#???##???? 2,4,1
?????.????? 3,3
??#?#?#??#????#?#? 2,1,2,1,1,3
#?????????#???????#? 1,3,11
..???#???????.? 2,6
??#???.?#??.? 1,2
..?#.?##?#??.. 2,2,1,1
??.????.?? 1,3,1
?????#?????#.# 1,5,1,1
..?#????#? 2,1
???#????#???.? 1,6,1
?.??#.???#. 2,1,1
..???.??#??. 3,4
???#??##?.???? 5,3,1
.??##?#?.???#?#? 5,7
#.?.???#???? 1,5,1
?????#??.??? 1,3,1
????????####??.##? 3,7,3
???????##???? 1,8,1
???..###??#??#???# 2,3,8
??##????#? 5,1
#??#?#?#?#.#?##.. 10,1,2
##?#?#?????#??? 7,1,2,1
??????????#?##????.? 2,11
##????????.#.?#?# 2,6,1,4
?#?#????#.?### 1,6,3
#?#?#?#..??? 1,1,3,1
???.#?#### 1,6
????????#?.???# 2,4,3
#?????###????.??#?? 12,3
??#.#???#?????.#? 1,5,1,1,1
??#?.???#?????#???? 3,1,5,1,1,1
????#?.????.???? 2,2,2
???#??????????.? 4,7
?.??###???###?#???#? 11,1,1
???????..?? 1,3,2
???#?#?#??#???# 9,1,1
????.????#? 2,4
??#???.?.?.??.?##? 2,1,1,1,1,4
?#.#..???#? 1,1,5
??????????????#???#? 2,3,8
????#?..#?.?##?#? 1,1,1,6
?#.##???..??#?#????# 1,2,2,1,5,2
????#?##?#.? 8,1
..#?#????#? 3,1
???.????.?????## 1,1,1,1,6
#??.????????##??? 1,1,4,4,1
#??#??????. 4,1
??????.???#?????.. 1,1,7
?#.?.????????##??? 1,1,2,2,2,1
????.?##????? 1,1,8
?###?.?#.?##?.# 3,1,2,1
????????.??? 2,3,1
?#?#...##?? 4,2
??#?.#??#.. 1,1,1
#?#.?????.??.. 3,1,2,1
?..?#??.?#? 2,2
#??????#??#?.#???### 4,5,3,3
#??#???????????? 1,2,6,2
?#?????..#?? 4,1,1,1
?.#???#??? 2,2
?#..?.??##???#.?? 1,5,1
???.?.?...?##? 2,1,4
#???.#???#??## 1,1,3,5
.????.?..????.#?? 1,1
??#??.????##?.?.? 1,1,1,4,1
#???.????? 1,1,4
????#.#???.#?? 4,4,2
?#??.#??.##?????..# 1,1,3,2,3,1
???????##???.???.? 2,6,1,1
?.???#?.?.???#???? 4,2
?.??.????. 1,2
#???#???#??????##? 10,2,3
?.??????##???#???? 1,4,10
????#???????.##?##?# 1,2,1,1,7
?????#????#?? 2,5,1
???..?????#??#??.?#. 1,6,3,2
???????#?? 1,3
.??.???.????# 2,3,1,1
????#.#?#####?? 4,8
?#???????????.?. 6,1,1,1
????.?#?##???#????? 2,11
.?.#.??????#??????? 1,1,1,1,7,1
????###.??.. 7,1
???#?.???? 4,1
????.?????#?. 1,6
?##?.????? 3,1,1
?#.?#?#?????.?? 2,7,1
#?#????#?#..????#??# 1,1,5,7
?.##?#????? 4,1
.??#?.###??. 3,4
#??##?????#??#??. 6,6
.#?#?????#?..#? 4,4,1
?.?.??.?#?###? 1,1,1,3
????.?????? 1,4
??????..?.?#?#? 1,1,1,1,4
????##???????#.?.? 5,1,3,1,1
???#??##??##???#???? 12,4
?.??##??#???.?#?. 1,4,2,1,3
???.???#???##???.?. 2,4,5,1,1
?#??..???##???.?? 3,5
?????#????#?.??#.? 1,1,3,3,1
??.????#..??.#?#?#?? 2,7
??##??.????#? 5,2,2
?????????????. 1,1,5
.????.##?# 4,2,1
??.??.?.?? 1,1,1
?#??.???.??.? 2,1,1,2
?.?#?????.?????#? 6,1,1
??##?????#???. 10,1
.?#???????##..#? 1,4,3,1
???#.???.??? 1,1,1,1
?.?#.???..#? 1,2,2,1
?.?#????#??????# 9,2
????#?#?.?##??????? 1,3,2,5,2
?.????##???#??? 1,7,1,1
.?#?.??.##???#??#? 3,2,3,1,1
.#??##????#???#. 6,1,1,2
?????.?.???? 3,1,1
??#??#####??#??.? 2,6,1,1,1
?.?.?#????#?#..? 1,4,4,1
??????..??????#??? 2,5,3
.#?#?#?.??.?#.. 5,2,2
###.??.??????##?# 3,1,9
??.?.####?..? 1,5
????.??.??##. 3,4
#???#??#.???.##??? 1,4,2,4
????.??????. 1,2,2
???#???#??.? 1,3,3,1
????#?#?#?.??? 5,1
#??.??#??????. 1,8
??????#??? 3,2
???##????#?.#?# 1,4,3,1,1
????..????.??#??## 3,1,1,7
???????##???? 2,2,4,1
???.??##?????#.??##. 1,4,1,1,1,2
#.????????????..?? 1,8,1
???..?##???#?? 2,3,3
.?????#..###?. 1,1,4
???.?.???????? 1,1,2,1
????#?#????? 1,1,5
???.?#?#??#?#? 1,2,1,5
.?.???.??#?###??# 1,1,7,2
.???#??.?#??#?? 4,3,2
?#.#??#?..???? 1,1,1,3
#???#????.#??????#. 1,4,1,5,1
..????#??#? 2,1,1
?#...??#??? 1,4
.####?????# 6,1
???#?#??????..#??? 4,3,3
#.???????.????#?? 1,3,1,5
???.???????. 1,7
??#.??#???##??????? 1,10
?????.???.??#??#??# 1,1,1,3,6,1
?.?#???..?# 5,2
??#????##???..?. 3,4,1,1
##?????.###?.? 3,1,4,1
?..#?#??.?????? 1,4,1,1,1
??...#??.??.?. 3,1
.????#?..?#?# 5,3
???#?????????.????.? 11,1,2
.??#???.?#?#????#??. 3,5,3
??#??#.????. 5,2
??????#?.???.?????? 1,2,3,2,1
?##???##?..#??#?? 2,3,4,1
.???#??#????#??????. 7,7
???#?..??# 2,1,1
.?#.???.?????? 2,2,3,1
.#?..#????? 2,2,3"""


count_arrangements(input_string)


7307

## Part 2

As you look out at the field of springs, you feel like there are way more springs than the condition records list. When you examine the records, you discover that they were actually folded up this whole time!

To unfold the records, on each row, replace the list of spring conditions with five copies of itself (separated by ?) and replace the list of contiguous groups of damaged springs with five copies of itself (separated by ,).

So, this row:

```
.# 1
```

Would become:

```
.#?.#?.#?.#?.# 1,1,1,1,1
```

The first line of the above example would become:

```
???.###????.###????.###????.###????.### 1,1,3,1,1,3,1,1,3,1,1,3,1,1,3
```

In the above example, after unfolding, the number of possible arrangements for some rows is now much larger:

- ???.### 1,1,3 - 1 arrangement
- .??..??...?##. 1,1,3 - 16384 arrangements
- ?#?#?#?#?#?#?#? 1,3,1,6 - 1 arrangement
- ????.#...#... 4,1,1 - 16 arrangements
- ????.######..#####. 1,6,5 - 2500 arrangements
- ?###???????? 3,2,1 - 506250 arrangements

After unfolding, adding all of the possible arrangement counts together produces 525152.

Unfold your condition records; what is the new sum of possible arrangement counts?

### Solution

In [8]:

def count_arrangements(input_string):
    lines = input_string.splitlines()
    records = [line.split(' ')[0] for line in lines]
    inputs = [list(map(int, line.split(' ')[1].split(','))) for line in lines]
    
    records = [(f"{record}?"*5)[:-1] for record in records]
    inputs = [numbers*5 for numbers in inputs]      

    combinations = []
    for record, numbers in zip(records, inputs):
        combinations.append(evaluate_record(record, numbers))
    
    return sum(combinations)

### Example

In [9]:
input_string = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""


count_arrangements(input_string)


525152

### Submission

In [10]:
input_string = """???#...#.?#??#?#? 1,1,1,8
#?.???????#????#???. 1,1,12
????.#??##??? 1,2,3
??#.#????#???#.??. 2,10,1
??????.?.???##?#?. 6,1,7
?#????????????###?? 1,3,9,1
?.??#?#?#.???? 7,1
.????????..??.#?. 5,2,2
.?.??#???.. 1,1,1
???#?????????#?? 2,1,3,2
??#?#?????. 4,2
?####???.???#? 6,1,2
#?.?#?#?????.?#.?.? 1,5,2,1,1,1
??.##?#?##??.??? 1,7,1
???.###?#?# 1,7
?????##?#..#? 1,3,1,1
??#??#?.???.?#??#? 1,1,1,2,1,2
?#?#?.???.?? 4,1,1
.?#?????#?? 3,1,2
?#??????????.??? 3,7,1
???????##??###???#?? 2,3,5,4
?.???#.??? 1,1,1
???.#?#?????.?#???#? 2,5,1,3,3
?.#?.#????.??## 1,2,1,3
?????????##?#??????? 1,5,9,1
?###????#??##?#... 9,4
????.#?#????..???. 5,1
????#??.##??#?#??.?? 1,2,5,3
..?????#.???.??##? 1,1,2,1,3
?#?????#??##.??# 12,1
?#??#?#.#????? 1,4,2,2
???#?...???????.? 3,2,4
??#??#?#?#?????#?.#. 1,1,12,1
???#??..?###. 5,4
?#.??#?#????????# 2,2,1,7
????.?????. 1,2,1
?.??????#??.???#.?? 1,1,2,1,4,1
???#??????.??????. 6,1,1,1,1,1
?#??#?#??##??? 1,4,5
?..??#??????.??. 1,4,1,1,2
?.??##???..??? 1,3,1,1
?##???.????#?? 3,3
??????.??#????? 1,1,5
?##.????#????.#??. 2,8,1
????#????#??# 1,10
???.?#??????#???.? 1,2,1,1,1,2
.?###.##??? 4,2
??.????#???#???###.? 1,15,1
?#????#?#???. 3,1,1,4
???????.????#???? 6,4
..??..???# 2,1
.?.?#?????..????..? 1,4,1,1,2,1
?#?.?.???????? 3,1,1,5
?###?.??#??.?###?# 4,3,6
..?#?.??#??##? 3,2,2
.?????#?????#?##?? 1,3,2,5
?#???#??.?.#?????? 8,2,3
??.??????#?#???? 1,2,5,1
?.??????#? 1,3,1
?...#??.?????#? 1,3,6
?.??.?..?#???? 1,1,1,4
.#??????.? 1,1
?????..?#????#.? 1,2,2,4,1
#.###?#????. 1,5,1,1
.???????#.???#?. 1,1,3,1,1
?#?#?.???????. 4,2
#????????#? 8,1
#?.?#????????? 1,2,6
?#??#????##?.???.? 5,1,3,2,1
????#????#???#?.???? 1,10,1,4
?##??????.???#?..?? 9,1,2,1
?#.??#???.. 1,1,2
.?#?????#.? 3,3,1
??.????.#??? 1,2
#??#??.????#??.?.??? 1,1,1,7,1,1
##????.???. 2,2
??##?.?##? 3,3
?#?..?.??..??.?? 1,1
?#????????? 3,1,2
?##??.???.?.?# 3,1,1,2
?.####???????? 1,4,1,1
?..#.???#????? 1,7
?.??.??##???? 1,1,4,1
.#?#?????#?.#??# 1,1,1,1,4
??.?????#????.??##. 8,4
????????.??.##? 1,1,2,1,3
??##???#???#?#??.? 6,1,3
?..??#???#??. 1,8
????????#????#??#??. 2,11,1
?.???????????#?#. 1,3,1,1,4
?#?#.??#.???#???##?? 1,1,2,11
??#?#???#?#??????? 11,1
.#.??#?????#? 1,6,1
..?????????#?. 3,2
????.??.#.?#?#?????. 1,2,1,1,4,2
#??????????##?.??? 5,1,1,3,1,1
????????..???? 3,2,3
??.????#??#???# 1,7,1
..??#.?????????? 1,5
?#?#???#??#??##?#??? 1,3,1,2,8
?????#.?#???.???# 2,1,4,1,1
?.?#??.??.????#??#? 1,8
??#??.?????? 3,3
?#?..??.?#? 2,1,3
.????##???. 1,6
???...?.????##?? 2,7
???..?###?#? 3,6
??.?#?#??###?????#? 1,4,8,1
.????#??#?# 1,6,1
?????.??.? 1,2
?#???##??.?..### 1,5,1,3
.???#???#??. 4,3
???#?.?#????. 1,5
?????..??.#?#?###??# 1,2,1,10
????#??????.?#?..#?? 5,2,1,3,2
#????#???????#???#.? 6,10,1
????#??????.?.?..?. 9,1
.??????##????.???#.? 2,8,2,1
.??##?.???#?# 1,3,1,3
????##??##??#?#. 1,11,1
???#?##.?.??? 4,1
?????#?#??? 1,6
?...????????.? 1,2
#??????##??..??? 2,1,4,1
.????????#??. 3,1,1
??#??????##..?.????? 3,6,3
??###??.??###?. 6,5
???.???#???.#?.?? 1,6,1,1
?????#??????? 1,6
.#??#?#?#??.??? 2,7,1,1
#??..??????#? 1,1,1,5
?#..?##??#?. 1,7
?#?#?#??..?.?. 7,1,1
.????????????.?##??? 1,1,8,6
??????####?..# 2,6,1
?????#???#???????? 10,1,1
??#??#???.??#??? 3,1,2,1,3
.#..?.?.?###??#...# 1,1,1,6,1
?#?#???.#??#????..? 1,3,4,1,1
.?##???#????#???? 3,2,4,1
###?..#??#??? 4,1,1,1
???.??#??#??. 1,7
????#?##??? 1,1,5
?.????????????##? 1,1,3,1,4
??#????#??. 1,4,1
.??#???#?.???#???#?? 8,6
##???#??.???#?.?? 6,1,1,1,2
#?#??#???#? 3,2,3
??#??????#??#?? 2,6,1
?.????#?#..??????.?? 6,2
.??.??.#?#?##? 1,1,1,4
????...?????????? 3,5
??#####.###???.?? 7,3,2
?##.??#??#???#??? 2,11
??????.??.?.???? 3,1,1,1,1
?.##?????##?? 2,4
?.???#?????.??????? 1,5,1,1,1,1
?????????#??? 2,9
?#???#?.#?.???.# 2,1,2,1,1
?#??????#?#? 1,1,1,3
?#??????#??.#????? 9,1,1,1,1
#???.?.?#????#???# 4,1,2,5,1
??#??????? 1,6
?????.?#?? 1,1,2
?????##.?.?# 1,3,1,1
???#?????#??# 1,2,2,4
.?#??????..?# 4,1,1
.#???.?.??##.???? 3,1,4,4
???.???#.????.#????? 3,5
#??#??#... 4,1
??.??#??#..???.?? 5,1
??.?#?????..##????? 1,6,2,1,1
..?#??#???????#?#? 8,5
?.??#??.?.??????. 3,6
.#??###???????##??. 1,6,5
??????#???#? 1,3,1,1
??#.?##?.?? 1,1,4
.??#???.?#????.?. 6,2,1,1
.??????#??#?? 6,2
#.?#??##?.??? 1,6,1
.?.##?????. 1,2,1
.?#?.????# 2,5
.????#?.#? 1,2,1
?.???.####???#.?. 1,8
?.#?????.??.???#?? 1,1,1,1,1,6
??#.?#???##??#. 3,9
??.#???????.???#???? 1,1,1,1,2,4
??.???????##? 2,2,3,2
.?.??#..??.. 1,1,1,2
????#?.???###?#???# 4,1,6,1,1
????#??????...???? 4,3,1
#?.#?.?????#??? 2,1,1,4
.????????#???#??.?#? 2,2,8,1
?????.##?? 2,3
##??.##???? 2,4
?????#??#?#?.#??.?? 10,3,2
?????..??????#?. 4,3,4
..?#.?#?#???. 1,5
..?.?##??? 1,2,2
#??#???##?? 1,3,3
??.#?????.????. 1,2
?##?????#?? 6,1,1
????#??###? 3,4
??????#???????#? 6,5,2
?#.??.?#?.#????#.? 1,1,2,1,4
?????#?#?###???##?? 1,5,3,6
#???#?#?#??????.?? 10,2
?.??#???.#.?#?.?#?? 1,5,1,2,3
?#??##?#.????.. 7,3
??#..??.?#?? 3,2,1
???#??.???.?? 5,1,1,1
??.??#.?#.??.??? 1,2,2,1,1
??.#???#?. 1,2,1
??.??##???. 1,5
?##?#????#??.???.# 3,1,5,1,1
??###?###?#????.???? 1,13,1,1
.???.??##.? 1,4
??????.??###??????? 2,5,5
.#?#?#???#?? 1,7
????#?#..#??.? 3,1,1,1
??#?.##?##?###???? 1,11
.?.#??????##???? 1,1,1,4,1
#??.##??#???#?#???#? 1,15
?????#???#?#?.? 1,8
?#???##??..?## 7,3
????????#? 1,4
##????#???????? 2,4,1,2
?#?##.?###??..?.? 4,4,1
??##???##?.??? 8,2
.???????#.?##??### 7,8
#????????????.##?? 6,1,2,3
?.??????#?????..#? 1,2,1,1,1,2
????#??..?.#???#???? 6,1,1,2,2
.??????#?.?#??.? 3,2,2
?????.??##? 3,1,4
.?.???##.##?### 5,6
?##?????##???#?.?.#? 3,1,8,1,2
????.?##.?? 3,2
?.??.??.??#?###??? 1,1,1,1,6
?.??????#?# 2,3
??#?#?.?????# 6,1,1,1
?.#?#??#?? 1,2,2
??????.#???# 2,1,3
?.#..???#. 1,1,1
#???.????? 4,1
??#?????##???.??? 10,2
??.??#.??? 1,1,1
??..???????#??? 1,1,1,1,2
#??.????#?? 1,2,1
.#?.#???.?.? 1,1,1,1
??????.##?#?. 3,5
?##???????.?##??#?? 3,2,1,8
??#?????.?#?#. 4,4
.???????#??#???.?# 12,1
.#??????.??#?.? 3,2,2,1
?????#????.??#???.?? 10,5
??.#..??#? 1,1,2
?????#?.??#? 2,2,2
?.#???.#????..??#? 1,1,1,2,2,2
?##??..#.#?.?#. 4,1,1,1
?????#.??.#??? 1,1,1,3
??????#??.???##??.?. 8,5
?#????#??????.??? 3,2,2,1
#??????????##..??.?? 1,1,1,5,1,1
##??#????????..?.?.? 13,1
.????.#??#??.?#??# 1,1,1,3,2,1
.?##??#????...?????. 10,5
##.???????????.#??? 2,1,6,1,2,1
.??#??#?.?#???? 3,1,4
.???.??????## 1,2,3
????????#??#?#?. 5,7
???.??#??#?. 2,2,1
..??????##. 3,2
??????#????????? 1,5,1
##?#???###?#?.. 10,2
???..???##?.?#..?? 2,1,3,1,2
?????##??? 1,5
?.????#?.??#? 1,5,4
..????#??.?.#?? 1,4,1
#????#?.?.. 6,1
.?.#??.?#???#???? 2,5
.?#..??.????? 2,2,2,2
???.??#????.?.??#??# 1,4,1,6
.#???????..#????? 5,5
##???.??#?? 2,1,5
?##?????#??? 5,1,3
??.#.#?????#??? 1,1,1,1,3
??.??.?#?? 1,1,4
?????.?#?? 1,3
????#?#????? 1,8
????????#?### 5,5
.??.?.??#?? 1,1,1
????????#???#?. 3,3,2,2
??.#???..?.?.?. 3,1
???????#.?????? 5,5
.???#?.????. 4,1,1
.??#??#??.?# 4,1,1
.????..?.?? 3,1,2
#??#?#?#####.?? 2,1,7,1
.?#??#???.? 2,1,2
#???#?#??#??.? 1,2,1,3
?#?#.???#? 1,1,4
??.????#?..??.?.?#? 1,1,4,2,1,2
#??????#??.???##??. 2,7,1,5
?#???.#.?????#?#? 4,1,6
###???.??..###?#?.? 5,2,6
??#?#??##?????# 1,1,5,4
.##??######???#?? 3,7,2,1
?#.?????.#?#?#???? 1,4,7
??.?#???##?????#. 1,3,6,1
??????#???????#??. 1,2,1,2,3
.???.?#?????.?.? 1,3,1,1,1
?.##??????? 1,3,2
..????.??? 1,1
..??#?.??? 4,1,1
?##???#.#? 3,2,2
???#?????.#???? 2,1,1,3,1
.??.?????..# 2,1,1,1
.????????#??? 1,3
??.???#??????? 2,4,3
?..?#?????.?##.# 7,3,1
??.?#?#????? 1,9
?#..???#?#??#.? 1,6
?#?#?????.#??. 8,2
??????????. 1,4
.?#????#?.#?## 2,1,1,4
#?#?###????.???????. 11,1,1
.#??####.?#??? 7,4
.???.#???#?###?? 2,1,1,4
#??????#?? 2,1,3
?.?.##?#?.?..#? 1,2,1,1,1
???#?#???. 4,4
?.???.??#??? 1,1,3,1
???????..## 2,1,2
?.???????.?? 1,1,5,1
#???????.?#?#??##?? 3,1,1,6
?.??.?.???. 2,2
#??##???#??.??. 5,2,1
????????#? 3,1,1
.?????#??##?.#?##??. 8,5
..??..????#?? 1,5
?????#.?#???##??? 1,1,2,8
???#....?# 1,2
?.??????#??#?????? 1,1,7,2,1
?##?.#??.. 2,2
?.?.?..??#?#??? 1,1,1,4,1
.##?.#.#??#??. 2,1,1,2
.????????#?? 2,2,3
?..#????.????#?#?#. 1,1,3,3,3,1
?.#?.?????#???????# 2,12
????##?#????#?#. 12,1
?.?#???????.? 1,2,1,1
##?.????.? 3,2,1
.??#?#???#.??.???? 6,1,2
.???????###?.????. 1,1,5,1,2
..#?????????????? 8,1,1,1
??.#??#?.?# 2,1,1
???????????#????? 2,6
?#.??.???. 1,3
?#???.?.??.??????? 1,1,1,2,6
??????#?????.?? 5,1,2,1
.#???.?.???#? 2,1,3
??##.??.???.??? 3,1,1,2
???.????#??#??##??? 3,1,11
????..##?..?? 3,2
??????????? 1,1,5
???.?#.?.?#?#??????# 3,1,1,8
.??.??#??? 2,6
##?..??????? 3,5
??????#????#.?#???# 1,1,3,1,3,1
?#?????.#?. 3,1,1
?????.??#??##??? 1,1,7,1
.????#??#? 1,4,1
#???#??#????.??? 2,9,2
?#????#??.???.?.? 1,6,1,1,1
????????.#? 2,1,1
???#??##.??#??#?. 7,4
#??.????.??????#? 3,1,1,2,2
?##?#??..?#. 3,1,1,2
.?.?????##???#.?. 3,6
#.?????### 1,2,3
????.??.?# 2,2,1
?????.????.#???#?.? 1,2,2,6,1
????????#?#.????? 4,2,1,1,2
?##????.#? 3,1,2
????.??????? 3,1,1,1
???????#?.? 4,4
.#.??????????. 1,3,4
?????#?.?#.? 2,3,2,1
?#.?????##??.???#? 1,7,1,1,2
#?????.????#?? 1,2,4
???????.????#?? 5,1,2
??.??.##?????.??#?? 1,6,5
???.?????..?##???? 2,6
???????###??##?#? 1,1,12
????????.? 1,1,1
??..???????#? 2,7
??#???.#??##?????? 4,1,5,1,1
.?.???????????#?? 2,5
?????##?.? 1,3
?????.#.??#??#?##?# 3,1,3,6
???.????#? 3,1,1
.??.???.#??.#?##???. 2,3,1,7
..?????###?#???????. 11,2
????????.???????#??# 1,5,3,2,1
??????.##?##?.? 2,5
??.??#???#???.?.? 3,5
?#?#.??#????#.??# 3,2,1,1,1
????#??.##?????.#?. 6,4,1,1
?.?.??.???? 1,2,2
#.?????????#? 1,8
.?????????#??????? 1,3,3,4
???.??????.#.??? 1,1,1,1,3
???.?#?#??? 3,4
????????.???##.??? 1,1,1,1,3,1
??.?.#.?.????.?.? 1,1
???###?##???# 1,6,3
?.?.????##??. 1,1,4
?.????.??.###???? 1,1,1,4,1
?#??????.?????#???. 1,1,8
??????#?.? 1,3
####??#?..#?????# 7,1,2
.????#??.#.???? 1,1,1,2,1
??..#??.?#?##? 3,6
?.??.??##???#????#? 5,7
??????#??.?.??? 3,2,2
?.???..?????#??.. 1,1,8
..?????##?? 2,3
??????.?????#??? 1,1,1,8
????#...?#.?? 1,3,2
??#??.??#??#?? 2,8
#???????..? 1,1,3
?#.??#.?#??# 1,3,5
#???????????###??#?# 2,1,1,1,9
.??##?.?????? 4,5
.????.??.?#. 1,2
#??????????#?..? 2,4,3,1
#?.???###?##????.?? 2,10
??#?..??????. 1,3
#??#?????#?? 1,3,5
??..?#..??.?? 1,2,2,1
?.??.????.# 1,4,1
#????.???? 2,1,2
.????#??#?????#.? 7,2
?.?.??????? 1,3,1
#?#?###??##?#???.#?? 11,2,2
#?..?????#???? 2,2,3
?.??????.????.?# 1,3,1,1,1
??????#?????.? 1,1,3,1
???#????.?#????. 4,1
##?####?#?#?.???? 11,2
??.#?.?.?#?.? 1,1
????#????###?#????? 2,1,11,1
#??#?.?#????#?#????# 4,2,1,5,1
??.???#.??#.#?#?#??# 1,2,1,1,1,8
?#?????????.?? 1,5,1,2
???#????#???#..? 4,1,2,1,1
???..??##?# 1,5
??.?.#?#??#???#?# 1,1,4,3,3
?#??#?##?#???? 5,4,1
????##??#? 4,1
#?.?????.#?.? 2,3,1,1
??.?#????????? 2,4,1,2
??.?.???.?????#? 1,1,2,2
??.????.??? 3,3
??.??????##???? 1,3
?#????.??##? 2,1,3
??#?##??#?#? 1,4,1,1
.?.#???.#.???#??#?? 1,1,1,1,8
???#.??..#??????.?? 3,1,5,1,1
?##??.#?.??.?? 2,2,2
???#?.???.?##?? 4,1,1,2
.?????????#. 3,1,3
???###?.#.#?#?.#? 6,1,4,1
????#???#?? 2,7
??#??#????#?? 7,3
..?.?????? 1,2,1
?#?#??#??. 1,4,1
??##????.? 5,1,1
.???#?##?#??? 1,1,2,4
??#???.?##??. 4,4
??##?#?#???.#?????. 4,1,4,3,1
.????##?????#?? 1,1,2,6
?.##????.#?#?.#?? 1,4,3,1
?????#?#?? 1,6
.#?##??????.?##.??. 6,1,3,1
??#?.?????.?#??.? 3,1,3
.?????###?? 2,4,1
?.?#?#.??...#..?? 1,4,1,1,1
??.????.??????#? 2,4,1,3
.????.?##?#?? 2,3,2
#?#?####?#???.#????? 12,5
??#.?????? 2,3,1
.#???#??#???#?#?..?? 1,1,1,7,1
??#????#???.#? 7,2,1
#?#?#..#???#??? 5,1,2,1
.?#?#####??.? 8,1
???#?#?.??#???.?# 6,3,2
.??#???#??? 1,2,3
???#???#??#? 2,3,1
?????.??????.?.?? 2,3,1,1
.?##????.????? 5,3
##?.#??#?. 3,2,2
???.#?##?#?#??.#?? 2,1,6,1,1
??#????.??????#?#?? 6,5,1
??#?#?...???# 4,1,1
??.?#??.?##.?.#??? 3,3,4
#?#????.??????#?? 7,1,4
.??????#??? 2,5
#?#?..????????????? 4,6,2,1
..#?#????#?? 3,4
#??#?#??????#??#?#? 10,7
?????.?#?#??? 1,6
??###???.???#?.## 6,2,1,2
????#??#?##. 3,1,4
#????#??#?????#?.?? 1,8,2,1,2
#.???#?????.? 1,4,1,1
#?.??.?????#?##.? 1,1,4,4,1
.??.???????? 1,1,1,1
??.?##?????.. 1,6
??#???.???? 1,1,1
.?.?.??##?????#?? 1,1,10,1
?????????????? 9,1
??#.#.??#???? 1,1,5,1
.???????#?#????.?.#? 5,4,2,2
.#??#??#?.? 1,5
?##?#??##??#??#?.??? 15,1
?#?#????????#?.?.# 3,3,2,1,1,1
???#??.?????#??.. 3,6
???#??..???#?.?.??? 1,2
##???#.?.. 3,1,1
??.??#?#?#???##?#?.? 2,1,1,1,9
.#????#?#????#?#?. 9,4
???#?###.##. 1,1,3,2
????#.?..???#?? 1,3
.#.????#???.#??? 1,6,1,1,1
??#.?.?#???.#?..#?. 2,1,2,1,1,2
?#????..?? 2,1
????.?#???#.???? 3,5,3
??????????##.???#.#? 1,3,1,2,4,1
?.?#.????#???##.? 1,8
#?.#????.? 1,3
.#???#.?#?????.?? 1,3,5,1
?.#????##?? 1,3,3
##?.?.#.??????#?? 3,1,1,4,1
????.??.?.????? 1,1,1,1,2
.?#?##.??#???? 4,6
?.?????.?#.. 2,1,1
????.?#??# 1,1,1
#.####??#?.#??# 1,8,1,1
.??.##???? 1,2,1
??..???.#. 2,1,1
?..????.??? 1,2
.??????#...?.????? 1,1
???.??#???##?? 1,3,4
????#??#?#??????##?? 1,5,1,1,5
?#???..???#???#??? 2,6
??????.??##?. 2,3
??#?????##??#.#?#??# 3,7,6
???.?????##?? 1,7
.?#??#?.?.?? 1,1,1,1
??.????#?? 1,2,1
???###.#???? 4,2,1
?.?#????.???????.. 3,1,6
???.??.#?#..??##. 1,1,1,1,4
????#????#.?##? 2,1,2,3
??#?##?#??.??????? 8,3
..#???#????.??? 1,3,1,2
??#?.?.???????.#.?? 3,1,1,5,1,1
?????#??.?? 1,2,1
????????..?#??#??.## 1,1,2,6,2
.#??????##??.?.??.?? 9,1,1,1
??##??#?????. 8,2
#.?.?????? 1,2
.????#?#???#..??# 2,7,3
?.?#?????????.#? 1,1,1,3,1
..#????.?????? 3,1,1,2
??#??#?.????.???.? 3,2,1,1,2
#???#.??.##??# 3,1,5
?.?.?#?##.???? 1,1,4,2
.?###..??#??????## 4,1,4
???#??????#. 2,4
??####????#?.??.. 7,1,1
#?.??????#? 1,2,1
.?##?#????????#?.??? 3,1,1,1,5,3
?.???#?##?.. 2,4
????#???..?##?# 1,1,1,3,1
??#??.#?.?????.? 4,1,4,1
???#.???.??. 4,1,2
?#?#...??#. 2,1,3
????#.?#..??. 1,2,2,2
?.#??#????#?##??? 12,1
??#??????#? 6,3
???.????#??? 1,1,1,2
???#.????.????? 1,2,5
??.?.??#??.??#??? 1,1,5,1,3
??#??????? 2,2
?.??#???????##??? 1,6,3
???#?#?.???#????#?? 7,1,8
.????????????? 3,1,2,1
?????##?#?#??.... 1,8,1
.?##???????#.???. 3,2
???.???????? 1,1,1,1
???#.?????????#?#..? 1,1,1,9,1
?##???#????.??#?#? 10,4
#??.????????? 3,2,1,1
?#?..???#?#?? 3,1,6
?##?#.?##?. 4,4
?#.#?..?#???? 1,2,3,1
??.##??#??#?#? 2,5,4
????##?#???.#.??... 8,1,1
##???????..?.?#??.? 2,4,2
?.?#.?#???. 1,4
.?#.??.????? 1,1,1,1
??????.???.?#?##??.? 2,4
???.????.??#??.?? 1,1,2,4,2
###?#??#.???????? 8,3
???..??#.????. 2,1,1
???.?##???.??????# 2,6,4
.?#?##.?#?????.?? 5,4,1
.??..??.?#? 2,2
.?#??????.##?#? 4,2,2,2
???##.?.?.#???. 2,1,1
?????#???#? 7,1
.#?.???#?.??# 1,3,1,1
?.#?..?#??#####?# 2,10
??#?##???#?#???#?#? 1,8,1,1,1,1
??????#.??#?###. 2,1,6
?#?#?#???#??##???# 1,3,9,1
?#???.????#.??#?#?? 1,1,2,2,3,3
?#?#.#.??.?#??????? 3,1,2,3,1,2
.?#?...?????#.??? 3,4,1,2
?????????#?#?.?# 1,1,1,5,1
##?#?#.???????#?? 6,4
?#??#?????????# 3,1,4,2
???#???#????.?? 1,6,1,1,1
?#.????##?.? 2,6
?.???##??.?#??#? 3,4
??#?????#.?#?. 2,1,1,3
?.?????#?#??#.?.#??? 11,2,1
????#???#?? 1,3,1
???.?#.???.???. 1,1,2,1,1
??????#?..??#? 5,2
.?.?#.?#??. 1,4
?#.??????##????.???? 2,7,1,1,1
?#?#?.?????. 3,1,1
???.##.?.#?. 3,2,1,2
..????????#.####??? 6,5
??#.??..?##?##????#? 1,1,9,1
?#???.??##?.??????. 5,4,3
.?...??#?????#.# 1,7,1,1
??????.?#??.?.? 5,2,1
???..??##?????? 2,5,1
???##?#??????.#.??. 7,2,1,1,1
?????????#?? 1,2,1,1
??.##??.?### 1,2,1,4
??####?#?###..?????? 1,10,1,1,1
??.??#???#??? 1,2,4
????..???#???. 1,6
??????????#? 8,1
#?#.?..????? 1,1,1,2
.?????##???#??? 12,1
##??#?..?#..? 2,2,1,1
###???????. 5,1,1
??.?.##?##??#?.?# 1,1,8,2
???####?.????#?#???# 1,5,10
??.??#???.?????#?#?? 1,1,4,6
???.#??#??..??.?# 2,4,1,2
.?.??##?###.???##? 8,2
.#??.?#.??#?? 3,1,3
????#????#??#??#??? 5,8
#..#???.???#???. 1,1,1,1,3
?.##?????.?#?????. 6,4,1
.????????.??#?? 2,1,1,1,1
.?.????????##?.?.?#? 1,1,5,3
??.#???????## 1,1,1,2
??????#??#???#??#??? 3,1,9,1
??.#??.???#?. 3,4
????.???##. 3,5
#??.??#????? 2,2,1
#?????#??.# 1,6,1
????.??#????.??. 1,7,1
##???????????. 6,1
.??#?????.????#???? 5,1,1,3,1
?????#??#??.?? 1,8
???#..??.???.. 1,2
???#?.#..????#?# 2,1,1,1,1
????.?.??#??#?? 2,1,7
.???#?##????#?? 2,1,4,1
??#?????#?? 4,1
.#????#?###?. 2,7
..????#??? 3,1,1
####???#..??. 6,1
?#???#???##????#?#?. 2,1,10
??#?#?#?.?????# 1,6,1,1,1
?????.##?#?#?#?.??#? 1,2,9,1
.?.????.?. 1,1,1
?????????? 1,3,1
#????#?????? 1,3,3
#.?#..???? 1,1,1
?#?????..????#.?? 1,2,4,1
#?##??????.??. 1,2,3,2
???#??#??. 5,1
?##???.?.??#.?# 3,1,1,1,2
?#?????????? 2,7
??#?#???#?#?????? 2,5,1,1,1
..#?..?.##?.? 2,2,1
???????.???.#?##.?.? 1,4,3,4,1,1
??##??###.# 7,1
?###?#??.#.??..?? 4,3,1,1,2
??????#??#?. 2,2,2
?#?##??????..#?? 6,2,3
?#?#???????..#.#?? 9,1,1,1,1
#?#?#?#????.?? 1,5,2,1
?????#??.?#? 5,1,1
??##???.?#??#.??.? 5,5,1,1
.?#?.?##?? 3,2
.???????#??..?#?? 8,1
?????.???# 4,1
?#?....???..??#? 3,2,1,1
???####?????.???#??? 1,5,1,1,1,1
???.??????...#??? 1,5,3
????#??.???????? 1,1,1,1,3
?#.???.#### 1,3,4
?.?#???#?#?#???.? 1,10,1
???.??#.?.? 2,1
??.??.??????.??.?? 1,1,3,1,1,1
.#??.?#????.#?###? 3,4,1,1,3
#?.???#?.?. 1,2,1
???#???.?? 4,2
????.#?#?#.?#####?# 3,1,3,6,1
..??????##?#?????#?? 2,1,5,2,4
#??.??#????#???#???? 1,15
???????????.? 1,5,1
?#??#???#?? 5,3
.???##????.?????.#. 1,4,1,1,1,1
?.?#?#??.?#?????? 6,2,2
.????????????? 7,1
...?.#???????#? 1,1,2,1
?????.?#?? 2,1
?#.??????? 2,2
#????#???##? 1,6,2
.?#?.?.???#?# 3,1,4
?#??#.??.?#?? 1,1,3
????#.?????#?##.?? 5,1,6,2
???..???????.##?? 2,6,2
..?#???.#?? 2,2
???##?.??#????????. 4,2,2
????#?????.?#??#. 1,5,2,1,2
##.??..#?..?#???? 2,2,1,3,1
??????#???#?#??? 5,7
.?#??.#??#???.#.#? 4,1,1,1,1,1
?.?????#?#?.?? 1,5
?????????#?????#?# 10,2,1,1
??????#??##????#??? 1,5,2,2,2,1
.??#?#????.??### 1,1,1,1,4
?????#?????#????? 4,2,2,1
??.?.?#??????#.??? 1,2,3,1,1
.??.?.???? 2,1,1
.??#????????? 1,2,3
???#?????#??#????#? 1,1,1,5,1,2
?????##???#?? 3,4,1
##?????????# 5,1,1,1
???#??#??????.? 4,1,2
????.??#?#?? 1,1,4
#??????.#???. 1,4,4
????##.??#???#?.??? 3,5
????.?#??#.##???? 1,1,4,2,3
#?#???#??????#. 1,6,2,1
???.???##?#? 1,1,4
?????#.???? 5,3
?#??.????#?????# 1,1,5,3
????#?.?#?.?.?. 1,1,2,1,1
???.???#?#?#??? 1,9
.?##??.??#???#???#? 3,3,6
?????##??.?? 1,6
#????????#####.? 2,1,7,1
#?..??.??#?? 2,1,1,3
?##?#???#???.#??? 5,3,1,1
.?????????????????. 1,1,2,1,1,4
?.???#??.?#?????? 4,5,1
?.????#??#?? 1,1,1,1
??????.#???# 3,1,3,1
#?#??..???##?#?.? 4,6
#??#?.????.?.? 5,1,1
?.?????.??#?#?##??? 1,1,1,10
???#?#.???##???.?.? 1,3,7,1,1
#?#.?#??.??? 3,3,3
.?#???#???.???# 7,3
?#????#??. 3,3
????####?#?.???? 2,5,1,2
?#???#??...??.? 2,2,2
?????????????#?###? 2,11
??.???????###?# 1,4,4,1
????#??.?.???.#???? 6,1,1,1,1,1
??#?#???????? 6,1
#??#?#????#???.? 9,4
?.#?#?#????????#??.? 8,1,3
.?.???#???????????? 1,7,1,2,2
?#???###????.??#?# 1,5,2,5
??#?????.??# 1,2,3,2
?##????.??.?# 4,1,2,1
#.?#?#???? 1,1,5
????.##??.?#??????? 3,4,4,3
?#.??????#??? 1,1,1,3
?#?.#?..??.??? 1,2,1,3
..???????? 3,3
?#????????. 1,3,2
.?#.???#.?? 2,2
?.??????????.?? 3,3,1
?#.??????..?#? 1,4,3
????????.??.#??? 6,2,2,1
?#???????.?????? 2,4,5
???.?#???. 1,1,2
??????#.?##??????#?# 1,5,3,6
??#?????#??.???#??. 4,2,1,5
.?#??.?.?. 2,1
#??#?.#?.#??.???? 5,1,1,1,1
???#????.#.? 2,1
??????#???.#.?#??##? 3,1,5
????#???????.# 9,1,1
????????#?##?#..#?. 1,2,3,2,1,2
??.?##?##.??????#? 2,2,2,4,2
###??#???#??? 3,1,3,1
.?.?.?#?.???#?# 1,1,3,1,1
.?..?#?????.???#??#? 1,2,1,1,2,3
?#.?????????#??#?#?? 2,1,14
?##.#???#?????? 2,1,1,1,1
???#.##???.? 3,5
???????.???#?? 4,1,2
??.#?????#??? 1,10
?#???##???? 2,4,1
?????.????? 3,3
??#?#?#??#????#?#? 2,1,2,1,1,3
#?????????#???????#? 1,3,11
..???#???????.? 2,6
??#???.?#??.? 1,2
..?#.?##?#??.. 2,2,1,1
??.????.?? 1,3,1
?????#?????#.# 1,5,1,1
..?#????#? 2,1
???#????#???.? 1,6,1
?.??#.???#. 2,1,1
..???.??#??. 3,4
???#??##?.???? 5,3,1
.??##?#?.???#?#? 5,7
#.?.???#???? 1,5,1
?????#??.??? 1,3,1
????????####??.##? 3,7,3
???????##???? 1,8,1
???..###??#??#???# 2,3,8
??##????#? 5,1
#??#?#?#?#.#?##.. 10,1,2
##?#?#?????#??? 7,1,2,1
??????????#?##????.? 2,11
##????????.#.?#?# 2,6,1,4
?#?#????#.?### 1,6,3
#?#?#?#..??? 1,1,3,1
???.#?#### 1,6
????????#?.???# 2,4,3
#?????###????.??#?? 12,3
??#.#???#?????.#? 1,5,1,1,1
??#?.???#?????#???? 3,1,5,1,1,1
????#?.????.???? 2,2,2
???#??????????.? 4,7
?.??###???###?#???#? 11,1,1
???????..?? 1,3,2
???#?#?#??#???# 9,1,1
????.????#? 2,4
??#???.?.?.??.?##? 2,1,1,1,1,4
?#.#..???#? 1,1,5
??????????????#???#? 2,3,8
????#?..#?.?##?#? 1,1,1,6
?#.##???..??#?#????# 1,2,2,1,5,2
????#?##?#.? 8,1
..#?#????#? 3,1
???.????.?????## 1,1,1,1,6
#??.????????##??? 1,1,4,4,1
#??#??????. 4,1
??????.???#?????.. 1,1,7
?#.?.????????##??? 1,1,2,2,2,1
????.?##????? 1,1,8
?###?.?#.?##?.# 3,1,2,1
????????.??? 2,3,1
?#?#...##?? 4,2
??#?.#??#.. 1,1,1
#?#.?????.??.. 3,1,2,1
?..?#??.?#? 2,2
#??????#??#?.#???### 4,5,3,3
#??#???????????? 1,2,6,2
?#?????..#?? 4,1,1,1
?.#???#??? 2,2
?#..?.??##???#.?? 1,5,1
???.?.?...?##? 2,1,4
#???.#???#??## 1,1,3,5
.????.?..????.#?? 1,1
??#??.????##?.?.? 1,1,1,4,1
#???.????? 1,1,4
????#.#???.#?? 4,4,2
?#??.#??.##?????..# 1,1,3,2,3,1
???????##???.???.? 2,6,1,1
?.???#?.?.???#???? 4,2
?.??.????. 1,2
#???#???#??????##? 10,2,3
?.??????##???#???? 1,4,10
????#???????.##?##?# 1,2,1,1,7
?????#????#?? 2,5,1
???..?????#??#??.?#. 1,6,3,2
???????#?? 1,3
.??.???.????# 2,3,1,1
????#.#?#####?? 4,8
?#???????????.?. 6,1,1,1
????.?#?##???#????? 2,11
.?.#.??????#??????? 1,1,1,1,7,1
????###.??.. 7,1
???#?.???? 4,1
????.?????#?. 1,6
?##?.????? 3,1,1
?#.?#?#?????.?? 2,7,1
#?#????#?#..????#??# 1,1,5,7
?.##?#????? 4,1
.??#?.###??. 3,4
#??##?????#??#??. 6,6
.#?#?????#?..#? 4,4,1
?.?.??.?#?###? 1,1,1,3
????.?????? 1,4
??????..?.?#?#? 1,1,1,1,4
????##???????#.?.? 5,1,3,1,1
???#??##??##???#???? 12,4
?.??##??#???.?#?. 1,4,2,1,3
???.???#???##???.?. 2,4,5,1,1
?#??..???##???.?? 3,5
?????#????#?.??#.? 1,1,3,3,1
??.????#..??.#?#?#?? 2,7
??##??.????#? 5,2,2
?????????????. 1,1,5
.????.##?# 4,2,1
??.??.?.?? 1,1,1
?#??.???.??.? 2,1,1,2
?.?#?????.?????#? 6,1,1
??##?????#???. 10,1
.?#???????##..#? 1,4,3,1
???#.???.??? 1,1,1,1
?.?#.???..#? 1,2,2,1
?.?#????#??????# 9,2
????#?#?.?##??????? 1,3,2,5,2
?.????##???#??? 1,7,1,1
.?#?.??.##???#??#? 3,2,3,1,1
.#??##????#???#. 6,1,1,2
?????.?.???? 3,1,1
??#??#####??#??.? 2,6,1,1,1
?.?.?#????#?#..? 1,4,4,1
??????..??????#??? 2,5,3
.#?#?#?.??.?#.. 5,2,2
###.??.??????##?# 3,1,9
??.?.####?..? 1,5
????.??.??##. 3,4
#???#??#.???.##??? 1,4,2,4
????.??????. 1,2,2
???#???#??.? 1,3,3,1
????#?#?#?.??? 5,1
#??.??#??????. 1,8
??????#??? 3,2
???##????#?.#?# 1,4,3,1,1
????..????.??#??## 3,1,1,7
???????##???? 2,2,4,1
???.??##?????#.??##. 1,4,1,1,1,2
#.????????????..?? 1,8,1
???..?##???#?? 2,3,3
.?????#..###?. 1,1,4
???.?.???????? 1,1,2,1
????#?#????? 1,1,5
???.?#?#??#?#? 1,2,1,5
.?.???.??#?###??# 1,1,7,2
.???#??.?#??#?? 4,3,2
?#.#??#?..???? 1,1,1,3
#???#????.#??????#. 1,4,1,5,1
..????#??#? 2,1,1
?#...??#??? 1,4
.####?????# 6,1
???#?#??????..#??? 4,3,3
#.???????.????#?? 1,3,1,5
???.???????. 1,7
??#.??#???##??????? 1,10
?????.???.??#??#??# 1,1,1,3,6,1
?.?#???..?# 5,2
??#????##???..?. 3,4,1,1
##?????.###?.? 3,1,4,1
?..#?#??.?????? 1,4,1,1,1
??...#??.??.?. 3,1
.????#?..?#?# 5,3
???#?????????.????.? 11,1,2
.??#???.?#?#????#??. 3,5,3
??#??#.????. 5,2
??????#?.???.?????? 1,2,3,2,1
?##???##?..#??#?? 2,3,4,1
.???#??#????#??????. 7,7
???#?..??# 2,1,1
.?#.???.?????? 2,2,3,1
.#?..#????? 2,2,3"""


count_arrangements(input_string)


3415570893842